In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Nov 20 13:44:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               8W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
# Colab 셀에 입력하여 실행
!pip install librosa h5py optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.0 MB/s eta 0:00:00


# 라이브러리 임포트

# 초기 환경 설정


In [13]:
import os
import json
import numpy as np
import librosa
from tqdm import tqdm
import h5py

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report,
    f1_score,
    precision_score,
    recall_score,
    accuracy_score,
)
from sklearn.utils.class_weight import compute_class_weight

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau  # 학습률 스케줄러 임포트
from torch.utils.data import Dataset, DataLoader

import random
import re
import gc
from typing import List, Tuple, Dict, Any

# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 일반 설정
BATCH_SIZE = 512  # 배치 크기 조정 (기존보다 증가)
EPOCHS = 500  # 에포크 수 설정
LEARNING_RATE = 1e-4  # 학습률 설정

# 데이터 관련 설정 (Google Drive 내 경로로 변경)
data_output_dir = "/content/drive/MyDrive/Sonus/Feature-based"  # 데이터 출력 디렉토리
metadata_file = os.path.join(data_output_dir, "metadata.json")  # 메타데이터 파일 경로
cache_dir = "/content/drive/MyDrive/Sonus/Feature-based"  # 캐시 디렉토리 설정
os.makedirs(cache_dir, exist_ok=True)
hdf5_file = os.path.join(cache_dir, "preprocessed_data.h5")  # 전처리된 데이터 저장 경로
n_mfcc = 40  # MFCC 계수의 수 (모델 아키텍처 간소화)
max_len = 174  # MFCC 벡터의 최대 길이

# 모델 및 로그 디렉토리 설정 (Google Drive 내 경로로 변경)
models_output_dir = "/content/drive/MyDrive/Sonus/Feature-based/models/light"  # 모델 저장 디렉토리
os.makedirs(models_output_dir, exist_ok=True)
logs_dir = "/content/drive/MyDrive/Sonus/Feature-based/logs"  # 로그 디렉토리
os.makedirs(logs_dir, exist_ok=True)

# ----------------------------- GPU 및 CPU 설정 -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 중인 디바이스: {device}\n")

# CPU 사용 제한 설정 (필요에 따라 조정 가능)
os.environ["OMP_NUM_THREADS"] = "4"
os.environ["OPENBLAS_NUM_THREADS"] = "4"
os.environ["MKL_NUM_THREADS"] = "4"
os.environ["VECLIB_MAXIMUM_THREADS"] = "4"
os.environ["NUMEXPR_NUM_THREADS"] = "4"
torch.set_num_threads(4)

# ----------------------------- 전처리 함수 정의 -----------------------------
def sanitize_name(name: str) -> str:
    """
    문자열에서 알파벳, 숫자, 언더스코어만 남기고 나머지는 제거합니다.

    Args:
        name (str): 원본 문자열.

    Returns:
        str: 정제된 문자열.
    """
    return re.sub(r"[^a-zA-Z0-9_]", "", name.replace(" ", "_"))


def load_metadata(metadata_path: str) -> List[Dict[str, Any]]:
    """
    메타데이터를 로드합니다.

    Args:
        metadata_path (str): 메타데이터 파일의 경로.

    Returns:
        List[Dict[str, Any]]: 메타데이터 리스트.
    """
    metadata = []
    with open(metadata_path, "r") as f:
        for line in f:
            metadata.append(json.loads(line))
    return metadata


def extract_mfcc(file_path: str, n_mfcc: int = 40, max_len: int = 174) -> np.ndarray:
    """
    오디오 파일에서 MFCC 특징을 추출합니다.

    Args:
        file_path (str): 오디오 파일의 경로.
        n_mfcc (int): 추출할 MFCC 계수의 수.
        max_len (int): MFCC 벡터의 최대 길이.

    Returns:
        np.ndarray: 고정된 크기의 MFCC 배열.
    """
    try:
        y, sr = librosa.load(file_path, sr=None, mono=True)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        if mfcc.shape[1] < max_len:
            pad_width = max_len - mfcc.shape[1]
            mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode="constant")
        else:
            mfcc = mfcc[:, :max_len]
        return mfcc
    except Exception as e:
        print(f"MFCC 추출 실패: {file_path}, 에러: {e}")
        return np.zeros((n_mfcc, max_len))


def preprocess_and_save_data(
    metadata: List[Dict[str, Any]], data_dir: str, hdf5_path: str
) -> None:
    """
    데이터 전처리 및 HDF5 파일로 저장합니다.

    Args:
        metadata (List[Dict[str, Any]]): 메타데이터 리스트.
        data_dir (str): 오디오 데이터가 저장된 디렉토리.
        hdf5_path (str): 저장할 HDF5 파일의 경로.
    """
    total_samples = len(metadata)
    with h5py.File(hdf5_path, "w") as h5f:
        X_ds = h5f.create_dataset(
            "X", shape=(total_samples, n_mfcc, max_len), dtype=np.float32
        )
        Y_list = []
        for idx, data in enumerate(tqdm(metadata, desc="전처리 중")):
            sample_path = os.path.join(
                data_dir, data["relative_path"], data["sample_name"]
            )
            mfcc = extract_mfcc(sample_path, n_mfcc, max_len)
            X_ds[idx] = mfcc
            instruments = data["instruments"]
            Y_list.append(instruments)
        Y_encoded_strings = [json.dumps(instr_list) for instr_list in Y_list]
        dt = h5py.string_dtype(encoding="utf-8")
        Y_ds = h5f.create_dataset(
            "Y", data=np.array(Y_encoded_strings, dtype=object), dtype=dt
        )


def load_preprocessed_data(hdf5_path: str) -> Tuple[np.ndarray, np.ndarray]:
    """
    전처리된 데이터를 HDF5 파일에서 로드합니다.

    Args:
        hdf5_path (str): HDF5 파일의 경로.

    Returns:
        Tuple[np.ndarray, np.ndarray]: 특징 배열 X와 레이블 리스트 Y.
    """
    with h5py.File(hdf5_path, "r") as h5f:
        X = h5f["X"][:]
        Y = h5f["Y"][:]
    return X, Y


def encode_labels(Y: np.ndarray) -> Tuple[np.ndarray, MultiLabelBinarizer, List[str]]:
    """
    레이블을 이진 벡터로 인코딩합니다.

    Args:
        Y (np.ndarray): 레이블 리스트 (JSON-encoded strings).

    Returns:
        Tuple[np.ndarray, MultiLabelBinarizer, List[str]]: 인코딩된 레이블 배열, 레이블 변환기, 모든 악기 리스트.
    """
    all_instruments = set()
    parsed_Y = []
    for instruments in Y:
        if isinstance(instruments, bytes):
            instruments = instruments.decode("utf-8")
        if isinstance(instruments, str):
            try:
                instruments = json.loads(instruments)
            except json.JSONDecodeError:
                instruments = instruments.split(",")
        if not isinstance(instruments, list):
            instruments = []
        instruments = [instr.strip() for instr in instruments if instr.strip()]
        all_instruments.update(instruments)
        parsed_Y.append(instruments)
    all_instruments = sorted(list(all_instruments))

    if not all_instruments:
        print("경고: 모든 악기 리스트가 비어 있습니다.")

    mlb = MultiLabelBinarizer(classes=all_instruments)
    Y_encoded = mlb.fit_transform(parsed_Y)
    return Y_encoded, mlb, all_instruments


def augment_batch(batch_X: torch.Tensor) -> torch.Tensor:
    """
    데이터 증강을 수행합니다.

    Args:
        batch_X (torch.Tensor): 배치의 MFCC 데이터. (channels, time, frequency)

    Returns:
        torch.Tensor: 증강된 배치의 MFCC 데이터.
    """
    # 데이터 증강 예시 (필요에 따라 수정)
    scale = random.uniform(0.8, 1.2)
    batch_X = batch_X * scale

    noise = torch.randn_like(batch_X) * 0.05
    batch_X = batch_X + noise

    # 주파수 마스킹
    freq_masking = random.randint(0, n_mfcc // 10)
    if freq_masking > 0:
        f0 = random.randint(0, max(n_mfcc - freq_masking - 1, 0))
        batch_X[:, :, f0 : f0 + freq_masking] = 0

    # 시간 마스킹
    time_masking = random.randint(0, max_len // 10)
    if time_masking > 0:
        t0 = random.randint(0, max(max_len - time_masking - 1, 0))
        batch_X[:, t0 : t0 + time_masking, :] = 0

    return batch_X

# ----------------------------- 데이터셋 클래스 정의 -----------------------------
class MFCCDataset(Dataset):
    """
    MFCC 데이터를 위한 PyTorch Dataset 클래스
    """

    def __init__(self, X: np.ndarray, Y: np.ndarray, augment: bool = False):
        """
        초기화 함수

        Args:
            X (np.ndarray): 특징 데이터.
            Y (np.ndarray): 레이블 데이터.
            augment (bool): 데이터 증강 여부.
        """
        self.X = X
        self.Y = Y
        self.augment = augment

    def __len__(self) -> int:
        """
        데이터셋의 길이를 반환합니다.

        Returns:
            int: 데이터셋의 길이.
        """
        return len(self.Y)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        인덱스에 해당하는 데이터를 반환합니다.

        Args:
            idx (int): 데이터 인덱스.

        Returns:
            Tuple[torch.Tensor, torch.Tensor]: 특징과 레이블 텐서.
        """
        x = self.X[idx]
        y = self.Y[idx]

        x = np.expand_dims(x, axis=0)
        x = x.transpose(0, 2, 1)  # (채널, 시간, 주파수)
        x = torch.from_numpy(x).float()

        if self.augment:
            x = augment_batch(x)

        mean = x.mean()
        std = x.std()
        x = (x - mean) / (std + 1e-6)

        y = torch.tensor(y, dtype=torch.float32)

        return x, y

# ----------------------------- 모델 정의 -----------------------------
class CNNModel(nn.Module):
    def __init__(self, num_classes: int, dropout_rate: float = 0.3):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding='same')
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding='same')
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding='same')
        self.pool3 = nn.MaxPool2d(2)
        self.flatten = nn.Flatten()

        # 동적으로 conv_output_size 계산
        example_input = torch.zeros((1, 1, max_len, n_mfcc))  # 입력 예시
        with torch.no_grad():
            example_output = self.pool3(self.conv3(self.pool2(self.conv2(self.pool1(self.conv1(example_input))))))
        conv_output_size = example_output.numel()

        self.fc1 = nn.Linear(conv_output_size, 128)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.output_layer = nn.Linear(128, num_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv1(x)
        x = torch.relu(self.pool1(x))
        x = self.conv2(x)
        x = torch.relu(self.pool2(x))
        x = self.conv3(x)
        x = torch.relu(self.pool3(x))
        x = self.flatten(x)
        x = self.dropout1(x)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.output_layer(x)
        return x




# ----------------------------- 모델 학습 및 평가 함수 정의 -----------------------------
def train_model(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    criterion,
    optimizer,
    scheduler,
    num_epochs: int,
    device: torch.device,
) -> None:
    """
    모델을 훈련합니다.

    Args:
        model (nn.Module): 학습할 모델.
        train_loader (DataLoader): 훈련 데이터 로더.
        val_loader (DataLoader): 검증 데이터 로더.
        criterion: 손실 함수.
        optimizer: 옵티마이저.
        scheduler: 학습률 스케줄러.
        num_epochs (int): 에포크 수.
        device (torch.device): 장치 (CPU 또는 GPU).
    """
    best_val_loss = float("inf")
    patience = 8  # 조기 종료를 위한 patience 감소
    trigger_times = 0
    scaler = torch.amp.GradScaler()  # 혼합 정밀도 학습을 위한 GradScaler

    print("모델 학습 시작...\n")
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        total_correct = 0

        # 훈련 단계
        train_bar = tqdm(
            train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] 훈련 중", leave=False
        )
        for batch_X, batch_Y in train_bar:
            batch_X = batch_X.to(device)
            batch_Y = batch_Y.to(device)

            optimizer.zero_grad()
            with torch.amp.autocast("cuda"):  # 혼합 정밀도 학습 적용
                outputs = model(batch_X)
                loss = criterion(outputs, batch_Y)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item() * batch_X.size(0)
            preds = torch.sigmoid(outputs) > 0.5  # 예측값 계산을 위해 sigmoid 적용
            total_correct += (preds == batch_Y.bool()).sum().item()

            # 배치별 손실 업데이트
            train_bar.set_postfix({"Loss": f"{loss.item():.4f}"})

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = total_correct / (len(train_loader.dataset) * batch_Y.size(1))  # 멀티레이블 정확도 계산

        # 검증 단계
        model.eval()
        val_loss = 0.0
        val_correct = 0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            val_bar = tqdm(
                val_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] 검증 중", leave=False
            )
            for batch_X, batch_Y in val_bar:
                batch_X = batch_X.to(device)
                batch_Y = batch_Y.to(device)

                with torch.amp.autocast("cuda"):  # 혼합 정밀도 적용
                    outputs = model(batch_X)
                    loss = criterion(outputs, batch_Y)
                val_loss += loss.item() * batch_X.size(0)
                preds = torch.sigmoid(outputs) > 0.5  # 예측값 계산
                val_correct += (preds == batch_Y.bool()).sum().item()
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(batch_Y.cpu().numpy())

        avg_val_loss = val_loss / len(val_loader.dataset)
        avg_val_acc = val_correct / (len(val_loader.dataset) * batch_Y.size(1))  # 멀티레이블 정확도 계산

        # F1-score, Precision, Recall 계산
        epoch_f1 = f1_score(all_labels, all_preds, average="samples", zero_division=0)
        epoch_precision = precision_score(
            all_labels, all_preds, average="samples", zero_division=0
        )
        epoch_recall = recall_score(
            all_labels, all_preds, average="samples", zero_division=0
        )

        # 에포크별 성능 지표 출력
        tqdm.write(
            f"Epoch [{epoch+1}/{num_epochs}] 완료 - "
            f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, "
            f"Val Loss: {avg_val_loss:.4f}, Val Acc: {avg_val_acc:.4f}, "
            f"Val Precision: {epoch_precision:.4f}, Val Recall: {epoch_recall:.4f}, Val F1-Score: {epoch_f1:.4f}"
        )

        # 에포크별 학습률 출력
        current_lr = optimizer.param_groups[0]['lr']
        tqdm.write(f"Epoch [{epoch+1}/{num_epochs}] 학습률: {current_lr}")

        # Early Stopping 체크 후 스케줄러 업데이트
        if avg_val_loss < best_val_loss - 1e-5:  # 손실이 1e-5 이상 개선되었을 때만 갱신
            best_val_loss = avg_val_loss
            trigger_times = 0
            save_model(
                model,
                os.path.join(
                    models_output_dir, f"best_model_light.pt"
                ),
            )
            tqdm.write(f"--> 최고 검증 손실 기록: {best_val_loss:.4f} 저장됨.\n")
        else:
            trigger_times += 1
            tqdm.write(f"--> EarlyStopping 트리거 횟수: {trigger_times}/{patience}\n")
            if trigger_times >= patience:
                tqdm.write("Early stopping 발생. 학습 중단.\n")
                break

        # 학습률 스케줄러 업데이트 (Early Stopping 체크 후에 수행)
        scheduler.step(avg_val_loss)

    tqdm.write(f"모델 학습 완료. 최고 검증 손실: {best_val_loss:.4f}\n")


def evaluate_model(
    model: nn.Module, test_loader: DataLoader, criterion, device: torch.device
) -> Tuple[float, float, List[float], List[float]]:
    """
    모델을 평가합니다.

    Args:
        model (nn.Module): 평가할 모델.
        test_loader (DataLoader): 테스트 데이터 로더.
        criterion: 손실 함수.
        device (torch.device): 장치 (CPU 또는 GPU).

    Returns:
        Tuple[float, float, List[float], List[float]]: 테스트 손실, 정확도, 예측 값 리스트, 실제 값 리스트.
    """
    model.eval()
    test_loss = 0.0
    test_correct = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch_X, batch_Y in test_loader:
            batch_X = batch_X.to(device)
            batch_Y = batch_Y.to(device)
            with torch.amp.autocast("cuda"):
                outputs = model(batch_X)
                loss = criterion(outputs, batch_Y)
            test_loss += loss.item() * batch_X.size(0)
            preds = torch.sigmoid(outputs) > 0.5  # 예측값 계산을 위해 sigmoid 적용
            test_correct += (preds == batch_Y.bool()).sum().item()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch_Y.cpu().numpy())

    avg_test_loss = test_loss / len(test_loader.dataset)
    avg_test_acc = test_correct / (len(test_loader.dataset) * batch_Y.size(1))  # 멀티레이블 정확도 계산

    return avg_test_loss, avg_test_acc, all_preds, all_labels


def save_model(model: nn.Module, path: str) -> None:
    """
    모델을 지정된 경로에 저장합니다.

    Args:
        model (nn.Module): 저장할 모델.
        path (str): 저장할 파일 경로.
    """
    torch.save(model.state_dict(), path)
    print(f"모델 저장 완료: {path}")


def load_model(model: nn.Module, path: str, device: torch.device) -> nn.Module:
    """
    모델을 지정된 경로에서 로드합니다.

    Args:
        model (nn.Module): 로드할 모델 구조.
        path (str): 모델 파일 경로.
        device (torch.device): 장치 (CPU 또는 GPU).

    Returns:
        nn.Module: 로드된 모델.
    """
    model.load_state_dict(torch.load(path, map_location=device))
    return model

def predict_instruments(audio_file_path: str, model: nn.Module, instrument_list: List[str], device: torch.device, threshold: float = 0.5) -> List[str]:
    """
    개별 음원의 악기를 예측합니다.

    Args:
        audio_file_path (str): 오디오 파일 경로.
        model (nn.Module): 학습된 모델.
        instrument_list (List[str]): 악기 이름 리스트.
        device (torch.device): 실행 장치.
        threshold (float): 예측 임계값 (default=0.5).

    Returns:
        List[str]: 예측된 악기 이름 리스트.
    """
    try:
        print(f"파일 로드 중: {audio_file_path}")
        # MFCC 추출
        mfcc = extract_mfcc(audio_file_path, n_mfcc=n_mfcc, max_len=max_len)
        if mfcc.shape != (n_mfcc, max_len):
            raise ValueError(f"MFCC 크기가 예상 범위를 벗어남: {mfcc.shape}")

        # 배치 및 채널 차원 추가 및 전치
        mfcc = np.expand_dims(mfcc, axis=0)  # 채널 추가
        mfcc = np.expand_dims(mfcc, axis=0)  # 배치 추가
        mfcc = mfcc.transpose(0, 1, 3, 2)   # (배치, 채널, 시간, 주파수)로 변환

        # 텐서 변환
        mfcc_tensor = torch.tensor(mfcc, dtype=torch.float32).to(device)

        # 모델 예측
        model.eval()
        print("모델 예측 중...")
        with torch.no_grad():
            outputs = model(mfcc_tensor)
            sigmoid_outputs = torch.sigmoid(outputs).cpu().numpy()[0]  # Sigmoid 적용
            print("Sigmoid Outputs:", sigmoid_outputs)

            # 임계값에 따라 이진화
            binary_predictions = sigmoid_outputs > threshold
            print("Binary Predictions:", binary_predictions)

            # 예측된 악기 이름 디코딩
            predicted_instruments = [
                instrument_list[i]
                for i, is_present in enumerate(binary_predictions)
                if is_present
            ]
            print("Predicted Instruments:", predicted_instruments)
        return predicted_instruments

    except Exception as e:
        print(f"예측 실패: {e}")
        return []


# ----------------------------- 메인 실행 부분 -----------------------------
if __name__ == "__main__":
    # 메타데이터 로드
    print("메타데이터 로드 중...")
    metadata = load_metadata(metadata_file)
    print(f"메타데이터 로드 완료: {len(metadata)} 샘플")

    # 데이터 전처리 및 저장
    if not os.path.exists(hdf5_file):
        print("데이터 전처리 시작...")
        preprocess_and_save_data(metadata, data_output_dir, hdf5_file)
        print("데이터 전처리 및 저장 완료.")
    else:
        print("전처리된 데이터 파일이 존재합니다. 로드합니다.")

    # 전처리된 데이터 로드
    print("전처리된 데이터 로드 중...")
    X, Y = load_preprocessed_data(hdf5_file)
    print(f"전처리된 데이터 로드 완료: X shape={X.shape}, Y shape={Y.shape}")

    # 레이블 인코딩
    print("레이블 인코딩 중...")
    Y_encoded, mlb, all_instruments = encode_labels(Y)
    print(f"레이블 인코딩 완료: {len(all_instruments)} 종류의 악기")

    if len(all_instruments) == 0:
        raise ValueError("레이블 인코딩 실패: 악기 리스트가 비어 있습니다.")

    # 데이터셋 분할
    print("데이터셋 분할 중...")
    X_train, X_temp, Y_train, Y_temp = train_test_split(
        X, Y_encoded, test_size=0.2, random_state=42
    )
    X_val, X_test, Y_val, Y_test = train_test_split(
        X_temp, Y_temp, test_size=0.5, random_state=42
    )
    print(f"데이터셋 분할 완료:")
    print(f" - 훈련 세트: {X_train.shape[0]} 샘플")
    print(f" - 검증 세트: {X_val.shape[0]} 샘플")
    print(f" - 테스트 세트: {X_test.shape[0]} 샘플")

    # 데이터셋 및 데이터로더 생성
    train_dataset = MFCCDataset(X_train, Y_train, augment=True)
    val_dataset = MFCCDataset(X_val, Y_val, augment=False)
    test_dataset = MFCCDataset(X_test, Y_test, augment=False)

    train_loader = DataLoader(
        train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True
    )
    val_loader = DataLoader(
        val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True
    )
    test_loader = DataLoader(
        test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True
    )

    # 레이블과 인덱스 확인
    for idx, instrument in enumerate(all_instruments):
        print(f"Index: {idx}, Instrument: {instrument}")

    # 모델 구축
    print("모델 구축 중...")
    # 입력 데이터 크기 확인
    input_shape = (1, max_len, n_mfcc)  # (채널, 시간, 주파수)
    num_classes = len(all_instruments)
    model = CNNModel(num_classes=num_classes).to(device)


    # 옵티마이저와 손실 함수 설정
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    criterion = nn.BCEWithLogitsLoss()  # BCEWithLogitsLoss로 변경

    # 학습률 스케줄러 설정
    scheduler = ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=0.5,
        patience=4,  # patience를 늘려 학습률 감소 전에 더 많은 에포크를 기다림
        verbose=True,
        threshold=1e-5,  # 손실 개선을 감지하기 위한 임계값을 작게 설정
        threshold_mode='abs'
    )

    # 모델 훈련
    # print("모델 훈련 시작...")
    # train_model(
    #     model=model,
    #     train_loader=train_loader,
    #     val_loader=val_loader,
    #     criterion=criterion,
    #     optimizer=optimizer,
    #     scheduler=scheduler,
    #     num_epochs=EPOCHS,
    #     device=device,
    # )
    # print("모델 훈련 완료.")

    # 최상의 모델 로드
    try:
        model = load_model(
            model,
            os.path.join(
                models_output_dir, f"best_model_light.pt"
            ),
            device,
        )
    except Exception as e:
        print(f"모델 로드 중 오류 발생: {e}")

    # 모델 평가
    print("모델 평가 중...")
    avg_test_loss, avg_test_acc, all_preds, all_labels = evaluate_model(
        model=model, test_loader=test_loader, criterion=criterion, device=device
    )
    print(f"테스트 손실: {avg_test_loss}")
    print(f"테스트 정확도: {avg_test_acc}")

    # F1-score 계산
    f1 = f1_score(all_labels, all_preds, average="samples", zero_division=0)
    print(f"F1-score: {f1}")

    # 분류 리포트 출력
    report = classification_report(
        all_labels,
        all_preds,
        target_names=all_instruments,
        zero_division=0,
    )
    print(report)

    # 예측 결과 저장
    print("사용자 평가를 위한 예측 결과를 저장합니다.")
    predictions_array = np.array(all_preds)  # shape=(samples, instruments)
    np.save(os.path.join(data_output_dir, "predictions.npy"), predictions_array)
    print("예측 결과 저장 완료: predictions.npy")



    # 테스트할 개별 음원 경로
    test_audio_path = "/content/drive/MyDrive/Sonus/Feature-based/concerto_sample_1.mp3"

    # 모델을 사용해 예측
    predicted_instruments = predict_instruments(
        audio_file_path=test_audio_path,
        model=model,
        instrument_list=all_instruments,
        device=device,
        threshold=0.5  # 임계값 설정
    )
    print(f"예측된 악기: {predicted_instruments}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
사용 중인 디바이스: cuda

메타데이터 로드 중...
메타데이터 로드 완료: 231980 샘플
전처리된 데이터 파일이 존재합니다. 로드합니다.
전처리된 데이터 로드 중...
전처리된 데이터 로드 완료: X shape=(231980, 40, 174), Y shape=(231980,)
레이블 인코딩 중...
레이블 인코딩 완료: 16 종류의 악기
데이터셋 분할 중...
데이터셋 분할 완료:
 - 훈련 세트: 185584 샘플
 - 검증 세트: 23198 샘플
 - 테스트 세트: 23198 샘플
Index: 0, Instrument: bass clarinet
Index: 1, Instrument: bassoon
Index: 2, Instrument: cello
Index: 3, Instrument: clarinet
Index: 4, Instrument: clash cymbals
Index: 5, Instrument: double bass
Index: 6, Instrument: flute
Index: 7, Instrument: french horn
Index: 8, Instrument: oboe
Index: 9, Instrument: saxophone
Index: 10, Instrument: tambourine
Index: 11, Instrument: trombone
Index: 12, Instrument: trumpet
Index: 13, Instrument: tuba
Index: 14, Instrument: viola
Index: 15, Instrument: violin
모델 구축 중...
모델 평가 중...


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
<ipython-input-13-1fce27d63a68>:530: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of 

테스트 손실: 0.34189200249455365
테스트 정확도: 0.8422681912233814
F1-score: 0.8182959508846781
               precision    recall  f1-score   support

bass clarinet       0.84      0.88      0.86     12396
      bassoon       0.88      0.89      0.88     12471
        cello       0.77      0.84      0.81     12391
     clarinet       0.82      0.88      0.85     12483
clash cymbals       1.00      0.99      0.99     12448
  double bass       0.86      0.88      0.87     12534
        flute       0.75      0.83      0.79     12412
  french horn       0.78      0.85      0.81     12567
         oboe       0.82      0.86      0.84     12488
    saxophone       0.84      0.85      0.84     12395
   tambourine       0.99      0.98      0.99     12391
     trombone       0.80      0.86      0.83     12381
      trumpet       0.82      0.85      0.83     12460
         tuba       0.90      0.92      0.91     12383
        viola       0.77      0.83      0.80     12371
       violin       0.78      0.83